<a href="https://colab.research.google.com/github/smwaingeni-ai/AgriX-AfricaDeepTech2025/blob/main/prototype/main_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# STEP 1: Install required packages
!pip install opencv-python-headless --quiet

# STEP 2: Import packages
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from google.colab import files
import shutil

# STEP 3: Clone GitHub repo (replace if it exists)
!rm -rf AgriX-AfricaDeepTech2025
!git clone https://github.com/smwaingeni-ai/AgriX-AfricaDeepTech2025.git
%cd AgriX-AfricaDeepTech2025

# STEP 4: Upload a leaf image
uploaded = files.upload()
uploaded_filename = list(uploaded.keys())[0]

# Move uploaded image into project folder
LEAF_IMAGE_PATH = "data/crops/sample_leaf.jpg"
os.makedirs(os.path.dirname(LEAF_IMAGE_PATH), exist_ok=True)
shutil.move(uploaded_filename, LEAF_IMAGE_PATH)

print(f"✅ Image uploaded and moved to: {LEAF_IMAGE_PATH}")

# STEP 5: Load the TFLite model
model_path = "prototype/tflite_model/crop_disease_model.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("📦 Model input shape:", input_details[0]['shape'])

# STEP 6: Define prediction function
def predict_crop_disease(image_path):
    img = Image.open(image_path).convert('RGB').resize((128, 128))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0).astype(np.float32)

    interpreter.set_tensor(input_details[0]['index'], img_array)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])[0]

    class_names = ['Healthy', 'Maize___Leaf_Spot', 'Tomato___Bacterial_spot']
    prediction = class_names[np.argmax(output)]
    confidence = np.max(output)

    return prediction, confidence

# STEP 7: Run the model on uploaded image
prediction, confidence = predict_crop_disease(LEAF_IMAGE_PATH)
print(f"🌾 Predicted class: {prediction} ({confidence * 100:.2f}% confidence)")


Cloning into 'AgriX-AfricaDeepTech2025'...
remote: Enumerating objects: 670, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 670 (delta 138), reused 102 (delta 102), pack-reused 515 (from 1)
Receiving objects: 100% (670/670), 24.59 MiB | 19.81 MiB/s, done.
Resolving deltas: 100% (276/276), done.
/content/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025/AgriX-AfricaDeepTech2025


Saving sample_leaf.jpg to sample_leaf.jpg
✅ Image uploaded and moved to: data/crops/sample_leaf.jpg
📦 Model input shape: [  1 128 128   3]
🌾 Predicted class: Healthy (35.29% confidence)
